In [ ]:
%load_ext autoreload
%autoreload 2

import pylupnt as pnt
import glob
import pickle
import os
from tqdm import tqdm
import numpy as np
from pylupnt.features import SuperPoint
import matplotlib.pyplot as plt
import numpy as np

import main_pnp_vo

output_dir = main_pnp_vo.output_dir
figures_dir = output_dir / "figures"
os.makedirs(figures_dir, exist_ok=True)

In [ ]:
# Load configs and poses from output directory
output_path = sorted(glob.glob(str(output_dir / "main_pnp_vo_results_*.pkl")))[-1]
with open(output_path, "rb") as f:
    output = pickle.load(f)
pnt.Logger.info(f"Loaded {len(output['runs'])} runs from {output_path}", "Main")

In [ ]:
gt_poses_tmp = output["gt_poses"]
bTw0 = pnt.invert_transform(gt_poses_tmp[0])
gt_poses = np.array([bTw0 @ pose for pose in gt_poses_tmp])

In [ ]:
dataset = pnt.datasets.Dataset.from_config(output["ds_config"])

In [ ]:
extractors = {
    k: pnt.FeatureExtractor.from_config(v)
    for k, v in main_pnp_vo.extractor_configs.items()
}
matchers = {
    k: pnt.FeatureMatcher.from_config(v) for k, v in main_pnp_vo.matcher_configs.items()
}

In [ ]:
idx = 180
m = int(np.ceil(len(extractors) / 2))
fig, axs = plt.subplots(2, m, figsize=(10, 5))

plt.sca(axs.flat[0])
plt.imshow(dataset[idx]["cameras"]["front_left"]["rgb"])
plt.title("RGB")
plt.axis("off")

img_extractors = ["ORB", "AKAZE", "BRISK", "SIFT", "SuperPoint"]

for i, extractor_name in enumerate(img_extractors):
    img_data = dataset[idx]["cameras"]["front_left"]
    feats = extractors[extractor_name].extract(img_data.rgb)
    plt.sca(axs.flat[i + 1])
    plt.imshow(img_data.rgb)
    plt.title(extractor_name)
    plt.scatter(feats.uv[:, 0], feats.uv[:, 1], color="cyan", s=1, zorder=10)
    plt.axis("off")
    plt.text(
        0.98,
        0.98,
        f"{len(feats)} features",
        color="white",
        fontsize=10,
        va="top",
        ha="right",
        transform=plt.gca().transAxes,
        bbox=dict(
            facecolor="black", alpha=0.4, edgecolor="none", boxstyle="round,pad=0.2"
        ),
    )
plt.tight_layout()
plt.savefig(figures_dir / "feature_extraction_all.pdf", dpi=300)
plt.show()

In [ ]:
import time

img_extractors = ["ORB", "AKAZE", "BRISK", "SIFT", "SuperPoint"]

times = {}

for i, extractor_name in tqdm(enumerate(img_extractors), total=len(img_extractors)):
    feats = extractors[extractor_name].extract(img_data.rgb)
    times_tmp = []
    for i in range(100):
        rgb = dataset[i]["cameras"]["front_left"]["rgb"]
        ts = time.time()
        feats = extractors[extractor_name].extract(rgb)
        te = time.time()
        times_tmp.append(te - ts)
    times[extractor_name] = times_tmp

import numpy as np

# Calculate mean and std for each extractor
means = []
stds = []
for ex in img_extractors:
    arr = np.array(times[ex])
    means.append(np.mean(arr * 1000.0))  # ms
    stds.append(np.std(arr * 1000.0))  # ms

In [ ]:
plt.figure(figsize=(4, 3), dpi=300)
x = np.arange(len(img_extractors))
plt.bar(x, means, yerr=stds, color="C0", alpha=0.7, capsize=4)
plt.xticks(x, img_extractors)
plt.ylabel("Time per image [ms]")
plt.title("Feature extraction time (100 iterations)")
plt.tight_layout()
plt.savefig(figures_dir / "feature_extraction_bench.pdf", dpi=300)
plt.show()

In [ ]:
from tqdm import tqdm

img_combinations = [
    ("ORB", "BruteForce"),
    ("AKAZE", "Flann"),
    ("BRISK", "Flann"),
    ("SIFT", "LightGlue"),
    ("SuperPoint", "SuperGlue"),
    ("SuperPoint", "LightGlue"),
]

idx = 180
img_data1 = dataset[idx]["cameras"]["front_left"]
img_data2 = dataset[idx + 1]["cameras"]["front_left"]


h, w = img_data1["rgb"].shape[:2]
ws = 10
sep = np.ones((h, ws, 3))

res_dict = {}

fig, axs = plt.subplots(3, 2, figsize=(6, 4), dpi=300)
for i, (e_name, m_name) in tqdm(
    enumerate(img_combinations), total=len(img_combinations)
):
    e_cfg = main_pnp_vo.extractor_configs[e_name]
    m_cfg = main_pnp_vo.matcher_configs[m_name]
    e_cfg_tmp, m_cfg_tmp = main_pnp_vo.process_config(e_name, e_cfg, m_name, m_cfg)
    if e_cfg_tmp is None or m_cfg_tmp is None:
        continue
    extractor = pnt.FeatureExtractor.from_config(e_cfg_tmp)
    matcher = pnt.FeatureMatcher.from_config(m_cfg_tmp)

    feats1 = extractor.extract(img_data1.rgb)
    feats2 = extractor.extract(img_data2.rgb)
    matches = matcher.match(feats1, feats2)

    # Check valid feature coordinates in image 1
    u1 = feats1.uv[:, 0].astype(np.int32).clip(0, w - 1)
    v1 = feats1.uv[:, 1].astype(np.int32).clip(0, h - 1)

    depth1 = img_data1.depth[v1, u1]

    # Convert to 3D world coordinates
    xyz_c1 = pnt.uv_to_xyz(feats1.uv, depth1, img_data1.intrinsics)
    xyz1_w = pnt.apply_transform(img_data1.world_T_cam, xyz_c1)

    # Project to image 2
    uv2_true, depth2 = pnt.xyz_to_uv(
        xyz1_w, img_data2.intrinsics, img_data2.world_T_cam, return_depth=True
    )

    # Filter valid projections
    valid_mask2: np.ndarray = (
        (depth1 > 0)
        & (depth2 > 0)
        & (0 <= uv2_true[:, 0])
        & (uv2_true[:, 0] < w)
        & (0 <= uv2_true[:, 1])
        & (uv2_true[:, 1] < h)
    )

    # Match distance
    gt_thresh = 0.01 * w
    dist_match = np.linalg.norm(
        uv2_true[matches.indexes[:, 0]] - feats2.uv[matches.indexes[:, 1]], axis=1
    )
    dists_normed = np.clip(dist_match / gt_thresh / 2.0, 0, 1)
    dists_normed = np.where(np.isnan(dists_normed), 1.0, dists_normed)
    match_colors = np.vstack(
        [dists_normed, 1 - dists_normed, np.zeros(len(dists_normed))]
    ).T

    # Precision and recall
    matched = np.zeros(len(feats1), dtype=np.bool)
    matched[matches.indexes[:, 0]] = True
    dist_feats1 = np.full(len(feats1), np.nan)
    dist_feats1[matches.indexes[:, 0]] = dist_match
    pos = (dist_feats1 < gt_thresh) & valid_mask2
    neg = (dist_feats1 >= gt_thresh) | ~valid_mask2
    tp = matched & pos
    fp = matched & neg
    fn = ~matched & pos
    prec = np.sum(tp) / (np.sum(tp) + np.sum(fp))
    rec = np.sum(tp) / (np.sum(tp) + np.sum(fn))

    print(f"Positive: {np.sum(pos)}, Negative: {np.sum(neg)}")
    print(f"TP: {np.sum(tp)}, FP: {np.sum(fp)}, FN: {np.sum(fn)}")
    print(f"Precision: {prec:.2f}, Recall: {rec:.2f}")

    res_dict[f"{e_name} + {m_name}"] = {
        "name": f"{e_name} + {m_name}",
        "n_left": len(feats1),
        "n_right": len(feats2),
        "n_matches": len(matches),
        "tp": np.sum(tp),
        "fp": np.sum(fp),
        "fn": np.sum(fn),
        "prec": prec,
        "rec": rec,
    }

    plt.sca(axs.flat[i])
    idxs = np.random.choice(len(matches), size=min(100, len(matches)), replace=False)
    plt.imshow(np.hstack((img_data1.rgb, sep, img_data2.rgb)))
    for j in idxs:
        i1 = matches.indexes[j, 0]
        i2 = matches.indexes[j, 1]
        pt1 = feats1.uv[i1]
        pt2 = feats2.uv[i2]
        plt.plot(
            [pt1[0], pt2[0] + w + ws],
            [pt1[1], pt2[1]],
            color=match_colors[j],
            linewidth=0.5,
        )
    plt.axis("off")
    plt.title(f"{e_name} + {m_name}")
    plt.xlim(0, 2 * w + ws)
    plt.ylim(h, 0)

plt.tight_layout()
plt.savefig(figures_dir / "feature_matching_all.pdf", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
import pandas as pd

# Update table after each entry (move after loop for efficiency if needed)
res_table = pd.DataFrame(res_dict.values())
res_table["prec"] = res_table["prec"].map("{:.2f}".format)
res_table["rec"] = res_table["rec"].map("{:.2f}".format)

res_table[["name", "n_left", "n_right", "n_matches", "prec"]]

In [ ]:
plt.figure(figsize=(5, 5), dpi=400)

gt_x = gt_poses[:, 0, 3]
gt_y = gt_poses[:, 1, 3]
plt.plot(gt_x, gt_y, label="Ground truth", color="black", linestyle="--")
plt.scatter(gt_x[-1], gt_y[-1], s=20, label="Final position", color="black")

failed = []
for run in output["runs"][:]:
    poses = run["poses"]
    e_name = run["extractor"]
    m_name = run["matcher"]
    label = f"{e_name} + {m_name}"
    if len(poses) > 1:
        xy = poses[:, :2, 3]
        plt.plot(xy[:, 0], xy[:, 1], label=label)
        plt.scatter(xy[-1, 0], xy[-1, 1], s=20)
    else:
        failed.append(label)

for label in failed:
    plt.scatter([], [], label=label, facecolor="none", edgecolor="none")

plt.xlabel("X [m]")
plt.ylabel("Y [m]")
plt.legend(fontsize=8, loc="upper left", ncol=2)
plt.axis("equal")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
cam_name = "front_left"

metrics_results = {}
for run in output["runs"]:
    run_name = run["extractor"] + " + " + run["matcher"]
    result = {
        "num_matches": [],
        "prec": [],
        "rec": [],
        "ea_t": [],
        "er_t": [],
        "ea_R": [],
        "er_R": [],
    }

    n_frames = len(run["poses"])
    for t in tqdm(range(1, n_frames), desc=f"{run['extractor']} + {run['matcher']}"):
        # Features and matches
        feats1: pnt.Features = run["features"][t - 1]
        feats2: pnt.Features = run["features"][t]
        matches: pnt.Matches = run["matches"][t]

        # Images
        img_data1: pnt.ImageData = dataset[t - 1]["cameras"][cam_name]
        img_data2: pnt.ImageData = dataset[t]["cameras"][cam_name]

        # Poses
        wTc1 = run["poses"][t - 1]
        wTc2 = run["poses"][t]
        wTc1_gt = gt_poses[t - 1]
        wTc2_gt = gt_poses[t]

        # Compute error
        c2Tc1 = pnt.invert_transform(wTc2) @ wTc1
        c2Tc1_gt = pnt.invert_transform(wTc2_gt) @ wTc1_gt
        ea_t = np.linalg.norm(c2Tc1_gt[:3, 3] - c2Tc1[:3, 3])
        er_t = ea_t / np.linalg.norm(c2Tc1_gt[:3, 3])
        ea_R = pnt.rotation_angle(c2Tc1[:3, :3] @ c2Tc1_gt[:3, :3].T) * pnt.DEG
        er_R = ea_R / pnt.rotation_angle(c2Tc1_gt[:3, :3]) / pnt.DEG

        # Convert to 3D world coordinates
        h, w = img_data1.rgb.shape[:2]
        u1 = feats1.uv[:, 0].astype(np.int32).clip(0, w - 1)
        v1 = feats1.uv[:, 1].astype(np.int32).clip(0, h - 1)
        depth1 = img_data1.depth[v1, u1]
        xyz_c1 = pnt.uv_to_xyz(feats1.uv, depth1, img_data1.intrinsics)
        xyz1_w = pnt.apply_transform(img_data1.world_T_cam, xyz_c1)

        # Project to image 2
        uv2_true, depth2 = pnt.xyz_to_uv(
            xyz1_w, img_data2.intrinsics, img_data2.world_T_cam, return_depth=True
        )

        # Filter valid projections
        valid_mask2: np.ndarray = (
            (depth1 > 0)
            & (depth2 > 0)
            & (0 <= uv2_true[:, 0])
            & (uv2_true[:, 0] < w)
            & (0 <= uv2_true[:, 1])
            & (uv2_true[:, 1] < h)
        )

        # Compute pairwise distances between uv2_true and feats2.uv
        all_dists = np.linalg.norm(uv2_true[:, None, :] - feats2.uv[None, :, :], axis=2)
        dist_true2feats = np.min(all_dists, axis=1)
        dist_feats2true = np.min(all_dists, axis=0)

        # Match distance
        gt_thresh = 0.01 * w
        dist_match = np.linalg.norm(
            uv2_true[matches.indexes[:, 0]] - feats2.uv[matches.indexes[:, 1]], axis=1
        )
        dists_normed = np.clip(dist_match / gt_thresh / 2.0, 0, 1)
        dists_normed = np.where(np.isnan(dists_normed), 1.0, dists_normed)
        match_colors = np.vstack(
            [dists_normed, 1 - dists_normed, np.zeros(len(dists_normed))]
        ).T

        # Precision and recall
        matched = np.zeros(len(feats1), dtype=np.bool)
        matched[matches.indexes[:, 0]] = True
        dist_feats1 = np.full(len(feats1), np.nan)
        dist_feats1[matches.indexes[:, 0]] = dist_match
        pos = (dist_feats1 < gt_thresh) & valid_mask2
        neg = (dist_feats1 >= gt_thresh) | ~valid_mask2
        tp = matched & pos
        fp = matched & neg
        fn = ~matched & pos
        prec = np.sum(tp) / (np.sum(tp) + np.sum(fp))
        rec = np.sum(tp) / (np.sum(tp) + np.sum(fn))

        # Add results
        result["num_matches"].append(len(matches))
        result["prec"].append(prec)
        result["rec"].append(rec)
        result["ea_t"].append(ea_t)
        result["er_t"].append(er_t)
        result["ea_R"].append(ea_R)
        result["er_R"].append(er_R)

    result = {k: np.array(v) for k, v in result.items()}
    metrics_results[run_name] = result

In [ ]:
result = metrics_results["SuperPoint + LightGlue"]
run = [
    run
    for run in output["runs"]
    if run["extractor"] + " + " + run["matcher"] == "SuperPoint + LightGlue"
][0]

text_kwargs = dict(
    fontsize=10,
    ha="left",
    va="top",
    bbox=dict(facecolor="white", alpha=0.5, edgecolor="none", boxstyle="round,pad=0.2"),
)


fig, axs = plt.subplots(2, 1, figsize=(5, 2.5), dpi=300)

plt.sca(axs[0])
plt.plot(result["ea_t"])
plt.xlim(0, len(result["ea_t"]))
plt.ylabel("Translation\n Error [m]")
text = f"Min: {np.min(result['ea_t']):.2f} m    "
text += f"Max: {np.max(result['ea_t']):.2f} m\n"
text += f"Mean: {np.mean(result['ea_t']):.2f} m  "
text += f"Std: {np.std(result['ea_t']):.2f} m"
plt.text(0.5, 0.95, text, **text_kwargs, transform=plt.gca().transAxes)
plt.ylim(0, 0.2)
plt.grid()

plt.sca(axs[1])
plt.plot(result["ea_R"])
plt.xlim(0, len(result["er_t"]))
plt.ylabel("Rotation\n Error [deg]")
text = f"Min: {np.min(result['er_t']):.2f}$^\\circ$   "
text += f"Max: {np.max(result['er_t']):.2f}$^\\circ$\n"
text += f"Mean: {np.mean(result['er_t']):.2f}$^\\circ$ "
text += f"Std: {np.std(result['er_t']):.2f}$^\\circ$"
plt.text(0.5, 0.95, text, **text_kwargs, transform=plt.gca().transAxes)
plt.ylim(0, 1.0)
plt.xlabel("Frame")
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(5, 2), dpi=400)

gt_x = gt_poses[:, 0, 3]
gt_y = gt_poses[:, 1, 3]
plt.plot(gt_x, gt_y, label="Ground truth", color="black", linestyle="--")
plt.scatter(gt_x[-1], gt_y[-1], s=20, label="Final position", color="black")

xy = run["poses"][:, :2, 3]

pnt.plot_colored_line_by_value(
    xy[:, 0],
    xy[:, 1],
    result["er_t"],
    cmap="GnYlRd",
    vmax=0.1,
    cbar_label="Translation Error [m]",
)
plt.xlabel("X [m]")
plt.ylabel("Y [m]")
plt.legend(fontsize=8, loc="upper left")
plt.axis("equal")
plt.grid(True)
plt.tight_layout()

In [ ]:
run = [
    run
    for run in output["runs"]
    if run["extractor"] + " + " + run["matcher"] == "SuperPoint + LightGlue"
][0]
result = metrics_results["SuperPoint + LightGlue"]

fig, axs = plt.subplots(1, 4, figsize=(10, 3), dpi=400)
for i in range(4):
    plt.sca(axs[i])
    plt.imshow(dataset[i + 15]["cameras"]["front_left"]["rgb"])
    plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
ds_config = pnt.load_config(
    {
        "inherit_from": "unreal.yaml",
        "basedir": "/home/shared_ws6/local_traverse/base_1",
        "preload": "none",
        "data_types": ["rgb"],
        "cameras": ["front_left", "left_left", "right_left", "back_left"],
    }
)

ds = pnt.datasets.UnrealDataset(ds_config)

import matplotlib.animation as animation

# Put cameras in canonical left-front-right-back order if present
wanted_order = ["left_left", "front_left", "right_left", "back_left"]
cameras = [cam for cam in wanted_order if cam in ds_config["cameras"]]

# Prepare figure: 1 row, 4 columns
fig, axs = plt.subplots(1, 4, figsize=(12, 3), constrained_layout=True)

n_frames = len(ds)


def update(frame_idx):
    for i, cam_name in enumerate(cameras):
        ax = axs[i]
        ax.clear()
        ax.imshow(ds[frame_idx]["cameras"][cam_name]["rgb"])
        # Clean title: left/front/right/back
        ax.set_title(cam_name.split("_")[0].capitalize())
        ax.axis("off")
    # (constrained_layout handles spacing)


ani = animation.FuncAnimation(fig, update, frames=n_frames, interval=50, repeat=False)

plt.close(fig)  # Close to avoid double inline display


# Save as MP4
ani.save("unreal_multicam_animation.mp4", writer="ffmpeg", dpi=200)

print(f"Animation saved as unreal_multicam_animation.mp4 ({n_frames} frames)")

In [ ]:
# Load configs and poses from output directory
run_types = ["base", "higher_elevation", "motion_blur", "no_lights"]

outputs = {}
datasets = {}
for run_type in run_types:
    output_path = sorted(
        glob.glob(str(output_dir / f"main_pnp_vo_results_{run_type}*.pkl"))
    )[-1]
    with open(output_path, "rb") as f:
        output = pickle.load(f)
    pnt.Logger.info(f"Loaded {len(output['runs'])} runs from {output_path}", "Main")

    gt_poses_tmp = output["gt_poses"]
    bTw0 = pnt.invert_transform(gt_poses_tmp[0])
    gt_poses = np.array([bTw0 @ pose for pose in gt_poses_tmp])

    config = output["ds_config"]
    config["preload"] = "none"
    dataset = pnt.datasets.Dataset.from_config(config)

    outputs[run_type] = output
    datasets[run_type] = dataset

extractors = {
    k: pnt.FeatureExtractor.from_config(v)
    for k, v in main_pnp_vo.extractor_configs.items()
}
matchers = {
    k: pnt.FeatureMatcher.from_config(v) for k, v in main_pnp_vo.matcher_configs.items()
}

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 2, figsize=(8, 6), dpi=300)
if len(datasets) == 1:
    axs = [axs]

run_display_names = {
    "base": "Baseline",
    "higher_elevation": "Higher Sun Elevation",
    "motion_blur": "Motion Blur",
    "no_lights": "No Lights",
}


for idx, (run_type, dataset) in enumerate(datasets.items()):
    # Assume RGB camera is always "front_left"
    rgb = dataset[63]["cameras"]["front_left"]["rgb"]
    ax = axs.flatten()[idx]
    ax.imshow(rgb)
    ax.set_title(run_display_names[run_type])
    ax.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

fig, ax = plt.subplots(figsize=(5, 5), dpi=300)

# Plot ground truth (from base run, all will be aligned identically)
first_run_output = next(iter(outputs.values()))
gt_poses_tmp = first_run_output["gt_poses"]
bTw0 = pnt.invert_transform(gt_poses_tmp[0])
gt_poses = np.array([bTw0 @ pose for pose in gt_poses_tmp])
plt.scatter(
    gt_poses[-1, 0, 3],
    gt_poses[-1, 1, 3],
    s=20,
    label="Final position",
    c="k",
    linestyle="--",
)
ax.plot(
    gt_poses[:, 0, 3], gt_poses[:, 1, 3], label="Ground Truth", c="k", lw=2, zorder=10
)

colors = ["tab:blue", "tab:orange", "tab:green", "tab:red"]

# For storing trajectory errors
traj_errors = {}

for idx, (run_type, output) in enumerate(outputs.items()):
    # Find SuperPoint + SuperGlue run
    sp_sg_run = None
    for run in output["runs"]:
        if run["extractor"] == "SuperPoint" and run["matcher"] == "SuperGlue":
            sp_sg_run = run
            break
    if sp_sg_run is None:
        continue

    # Align estimated poses with their own first pose
    est_poses = np.array(sp_sg_run["poses"])
    if est_poses.shape[0] == 0:
        continue
    bTw0_est = np.linalg.inv(est_poses[0])
    est_poses_aligned = np.array([bTw0_est @ pose for pose in est_poses])
    display_name = run_display_names.get(run_type, run_type)
    ax.plot(
        est_poses_aligned[:, 0, 3],
        est_poses_aligned[:, 1, 3],
        label=f"{display_name}",
        lw=2,
        color=colors[idx % len(colors)],
        alpha=0.8,
        zorder=idx + 1,
    )
    plt.scatter(est_poses_aligned[-1, 0, 3], est_poses_aligned[-1, 1, 3], s=20)

    # Compute trajectory errors (translational and rotational)
    # - Use ground truth of the same length as estimated poses
    # - Both are aligned so direct comparison is valid
    trans_errors = []
    rot_errors = []
    n_frames = min(est_poses_aligned.shape[0], gt_poses.shape[0])
    for t in range(n_frames):
        est = est_poses_aligned[t]
        gt = gt_poses[t]
        # Translational error (Euclidean norm of translation difference)
        t_error = np.linalg.norm(est[:3, 3] - gt[:3, 3])
        # Rotational error (angle in degrees)
        R_diff = est[:3, :3] @ gt[:3, :3].T
        angle_error = pnt.rotation_angle(R_diff) * pnt.DEG
        trans_errors.append(t_error)
        rot_errors.append(angle_error)
    # Store errors for analysis
    traj_errors[run_type] = {
        "translational": np.array(trans_errors),
        "rotational": np.array(rot_errors),
    }

ax.set_xlabel("X [m]")
ax.set_ylabel("Y [m]")
ax.axis("equal")
ax.grid(True)
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Print mean/median trajectory errors for each run
print("Trajectory errors (SuperPoint+SuperGlue):")
for run_type, errors in traj_errors.items():
    t_err = errors["translational"]
    r_err = errors["rotational"]
    print(
        f"{run_display_names.get(run_type, run_type)}:"
        f"  mean t_err = {np.mean(t_err):.3f} m,"
        f"  median t_err = {np.median(t_err):.3f} m,"
        f"  mean r_err = {np.mean(r_err):.2f} deg,"
        f"  median r_err = {np.median(r_err):.2f} deg"
    )

import matplotlib.pyplot as plt

# Gather errors and display names
run_names_plot = [run_display_names.get(rt, rt) for rt in traj_errors.keys()]
t_errs = [errors["translational"] for errors in traj_errors.values()]
r_errs = [errors["rotational"] for errors in traj_errors.values()]

fig, axs = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
# Plot translational error (t_err)
for i, (t_err, name) in enumerate(zip(t_errs, run_names_plot)):
    axs[0].plot(t_err, label=f"{name}")
axs[0].set_ylabel("Trans. Error [m]")
axs[0].set_title("Translational Error")
axs[0].legend()
axs[0].grid(True)

# Plot rotational error (r_err)
for i, (r_err, name) in enumerate(zip(r_errs, run_names_plot)):
    axs[1].plot(r_err, label=f"{name}")
axs[1].set_ylabel("Rot. Error [deg]")
axs[1].set_xlabel("Frame")
axs[1].set_title("Rotational Error")
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Load 10 frames from the dataset
ds_long_config = pnt.load_config(
    {
        "inherit_from": "datasets/unreal.yaml",
        "step": 1,
        "basedir": "/home/shared_ws6/local_traverse/base_2",
        "agent": "rover0",
        "preload": "none",
        "data_types": ["rgb"],
        "cameras": ["front_left"],
    }
)
ds_long1 = pnt.datasets.Dataset.from_config(ds_long_config)

ds_long_config["agent"] = "rover1"
ds_long_config["cameras"] = ["back_left"]
ds_long2 = pnt.datasets.Dataset.from_config(ds_long_config)

frames1, frames2 = [], []
for i in range(500):
    img1 = ds_long1[i]["cameras"]["front_left"]["rgb"]
    img2 = ds_long2[i]["cameras"]["back_left"]["rgb"]
    frames1.append(img1)
    frames2.append(img2)

fig, axs = plt.subplots(1, 2, figsize=(8, 4), constrained_layout=True)
im1 = axs[0].imshow(frames1[0])
axs[0].axis("off")
axs[0].set_title("Rover 1 - Front")
im2 = axs[1].imshow(frames2[0])
axs[1].axis("off")
axs[1].set_title("Rover 1 - Back")


def animate(i):
    im1.set_array(frames1[i])
    im2.set_array(frames2[i])
    return [im1, im2]


ani = animation.FuncAnimation(
    fig, animate, frames=len(frames1), interval=300, blit=True
)

plt.close(fig)

# Save the animation as an MP4 file
ani.save("dataset_animation.mp4", writer="ffmpeg", fps=10)